## This notebook shows how to develop KNN Classifier for MNIST handwritten digits classification

In [1]:
from tkinter import * 
import tkinter as tk
from PIL import Image, ImageGrab, ImageOps
import numpy as np
import joblib

## Load Trained Model

In [2]:
classifier = joblib.load('/Users/daveyap/Desktop/github/Handwritten-digits-classification/mnist_knn_model.sav')

/Users/daveyap/Documents/project_folder/daveyap_python/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## GUI

In [3]:
class GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.canvas_width = 200
        self.canvas_height = 200
        self.bg_color = "White"
        self.paint_color = "black"
        self.radius = 5
        self.init_canvas()
        
    def init_canvas(self):
        self.canvas = Canvas(self, width=self.canvas_width, height=self.canvas_width, bg = self.bg_color, cursor="cross")
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        self.button_predict = tk.Button(self, text = "Predict", command = self.predict)
        self.label_digit = tk.Label(self, text="", font=("Helvetica"))
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.button_clear.grid(row=1, column=0, pady=2)
        self.button_predict.grid(row=1, column=1, pady=2)
        self.label_digit.grid(row=0, column=1, padx=2, pady=2)
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        x1, y1 = (event.x - self.radius), (event.y - self.radius)
        x2, y2 = (event.x + self.radius), (event.y + self.radius)
        self.canvas.create_oval(x1, y1, x2, y2, fill=self.paint_color, outline=self.paint_color)

    def clear_all(self):
        self.canvas.delete("all")
    
    def preprocess(self):
        fileName = 'canvas_paint'
        
        # Find Canvas Coordinates - I tweaked a little bit as a workaround
        x = self.canvas.winfo_x() + self.winfo_x() + 52
        y = self.canvas.winfo_y() + self.winfo_y() + 77
        x1 = x + self.canvas_width
        y1 = y + self.canvas_height
        
        # Screenshot and Save Image in Canvas
        img = ImageGrab.grab().crop((x,y,x1,y1))
        img.save(fileName + ".jpg")
        
        # Invert Image
        img = ImageOps.invert(img)
        img.save(fileName + "_negative" + ".jpg")
        
        # Resize and convert it into greyscale
        img = img.resize((28,28))
        img = img.convert('L')
        img.save(fileName + "_28x28" + ".jpg")
        
        # Convert it into Numpy array
        img = np.array(img)
        img = img.reshape(28*28)
        print("Pre Processing complete")
        return img
    
    def predict(self):
        img = self.preprocess()
        digit = classifier.predict([img])
        self.label_digit.configure(text=str(digit))

In [4]:
app = GUI()
mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/daveyap/Documents/project_folder/daveyap_python/lib/python3.7/site-packages/PIL/JpegImagePlugin.py", line 610, in _save
    rawmode = RAWMODE[im.mode]
KeyError: 'RGBA'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-e5d0ebc4b4fb>", line 59, in predict
    img = self.preprocess()
  File "<ipython-input-3-e5d0ebc4b4fb>", line 41, in preprocess
    img.save(fileName + ".jpg")
  File "/Users/daveyap/Documents/project_folder/daveyap_python/lib/python3.7/site-packages/PIL/Image.py", line 2164, in save
    save_handler(self, fp, filename)
  File "/Users/daveyap/Documents/project_folder/daveyap_python/lib/python3.7/site-packages/PIL/JpegImagePlugin.py", line 612, in _save
    raise OSError(f

I had to convert the drawing on the Canvas to an image that can be fed to the Machine Learning model. 

### Procedure of GUI

1.Drawing on the Canvas should be retrieved.Coordinates of the Canvas were calculated and passed to the ImageGrab method as parameters, to accomplish this task.
2.Invert the grabbed Image. The ML model is trained on images with Black background and white font. Whereas the grabbed image has white background and black font.
3.Resize the Image to 28*28. The ML model requires the input image to be in 28*28 dimensions, as it was trained only with 28*28 images.
4.Convert to Greyscale. Convert the image from RGB to Greyscale.
5.Store the image as Numpy data for further processing.